## Introduction
The following code solves the LASSO problem in the Fourier Domain.

The LASSO:
$\|\mathbf{A} \mathbf{c} - \mathbf{b} \|_2^2 + \lambda \|\mathbf{c} \|_1$

This is a toy example on how large dictionaries $\mathbf{A}$ can be distrubted over multiple GPUs. In general tensorflow is significently slower than many other optinos but it is particularly handy for prototyping. This is only a demonstration on the applicability of solving such large sclae problems in a GPU distributed fashion.

Refer to "FFTLasso: Large-Scale LASSO in The Fourier Domain" for more details.

## Dependencies

In [1]:
import tensorflow as tf
import numpy as np
import scipy.io as spio
import time

## Data and Regularization Parameter Initialization

Read data from disk, or simply generate your own gaussian dictionary (A) and test sample (b). Also, the choice of the regularization parameter is set ehre.

In [2]:
# #Reading data from disc
# matA = spio.loadmat('A.mat')
# matb = spio.loadmat('b.mat')
# A = matA['A']
# b = matb['b']
# m,n = A.shape
# print(m,n)

a = np.arange(20) + 1
b = np.arange(21,41)
A = np.array([a,b])
b = np.array([[1],[2]]).astype(np.float32)
m, n = A.shape

reg_param = np.array(1e-2).astype(np.float32) #regularization \lambda

## Optimization parameters

In here we initilize the ADMM parameters:

1) Maximum number of iterations (max_iters)

2) Stopping criterion based on the standard deviation of the last 5 objective values (threshold)

3) Update of the dual variables using nesterovs method (Q, init_rho)

4) The amound at which the augmentation constant is increased learning_rate (plays role in overall ADMM iteration count)

5) The constants regarding nesterovs update and how much is it updated in every iter (init_gamma_val, gamma_factor)

6) Every how many iterations do we carry out the updates to rho, init_gamma_cal (inc_iter)

7) Maximum permittable rho (upper_limit_rho)

In [3]:
def opt_parameters():
    max_iters = 1000 #max num iterations 
    threshold = 1e-8  #stopping criterion
    #Nesterov's accelaration
    Q = 30;
    init_qfac=np.array((np.sqrt(Q)-1)/(np.sqrt(Q)+1)).astype(np.float32) 
    init_rho  = np.array(1).astype(np.float32) #initial rho
    learning_rate = np.array(1).astype(np.float32) #increasing rate of rho
    upper_limit_rho = 2000
    inc_iter = 1     #upper bound on rho
    init_gamma_val = np.array(1).astype(np.float32)  #initial gamma value
    gamma_factor = 1 #increasing rate of gamma
    
    return (max_iters, threshold, init_qfac, init_rho, learning_rate, init_gamma_val, 
            gamma_factor, inc_iter, upper_limit_rho)

## Class of the FFTLasso solver

The class FFTLasso has the following methods:

1) _initialize: initializes the variables in the optimization along with the overhead data that is used only once.

5) fill_feed_dict: collects all the variables output of the optimization into a dictioray to be fed again to the network

3) compute_cost: computes the objective value

2) _graph: constructs the bulk of the graph for the optimization

4) solve: solves the optimization




In [4]:
class TFSolver(object):
    def __init__(self, gpus=1):
        self._gpus = gpus
        
################################################################
                    #Initlize the variables
################################################################

    def _initialize(self, A, b, rho):
        #break A_into piaces
        A_all = self.break_A_pieces(A,self._gpus)
        n_new = int(A.shape[1]/self._gpus)
        #dual variable intialization in Foureri Domain
        b_hat_conj = (np.fft.ifft(b.T).T).astype(np.complex64)
        init_psi_hat_conj = (np.fft.ifft(np.zeros([m,1]).T).T).astype(np.complex64)
        A_hat = [None] * self._gpus
        A_hat_conj = [None] * self._gpus
        corr_A_lists = [None] * self._gpus
        init_theta = [None] * self._gpus
        init_gamma = [None] * self._gpus
        temp = [None] * self._gpus
        init_zeta = [None] * self._gpus
        
        for i in range(self._gpus):
            #Data related initialization
            A_hat[i] = (np.fft.fft(A_all[i].T).T).astype(np.complex64)
            A_hat_conj[i] = (np.fft.ifft(A_all[i].T).T).astype(np.complex64) # = np.conj(A_hat)/m
             #constraints variable intilization
            init_theta[i] = (np.zeros([m*n_new,1])).astype(np.float32)
             #primal variable inilization
            init_gamma[i] = (np.zeros([m*n_new,1])).astype(np.float32)
            #init zeta
            temp[i] = np.real(np.reshape(np.fft.fft((A_hat[i]*init_psi_hat_conj).T).T,(m*n_new,1))).astype(np.float32)
            init_zeta[i] = (temp[i] + init_theta[i] + (init_gamma[i]))
            
        init_gamma_out = (np.zeros([n,1])).astype(np.float32)
        mask_update = np.ones([m,n]).astype(np.float32)
        mask_update[0] = 0
        mask_update = np.split(mask_update,self._gpus,1)
        corr_A = np.array(np.zeros([m,1]).T).T
        for i in range(self._gpus):
            corr_A = corr_A + np.reshape((np.real(np.sum(A_hat[i]*A_hat_conj[i],1))),(-1,1))
            mask_update[i] = np.reshape(mask_update[i].T,(-1,1))
            
        
        if(self._gpus > 1):
            e = np.array(init_zeta)-np.array(init_theta)-np.array(init_gamma)
            e_hat_conj  =(np.fft.ifft(np.reshape(e,(n,m)))).T  
            ccc = np.array(A_hat_conj)
            Ahat_conj_complete = ccc.reshape(ccc.shape[1],ccc.shape[0]*ccc.shape[2])
            upper = np.reshape(np.sum(Ahat_conj_complete* np.array(e_hat_conj),axis=1),(m,1)) + b_hat_conj
            lower = (rho * np.array(corr_A)) + 1.0
            init_psi_hat_conj = upper / lower 
        else:
            e = np.array(init_zeta)-np.array(init_theta)-np.array(init_gamma)
            e_hat_conj  =(np.fft.ifft(np.reshape(e,(n,m)))).T  
            upper = np.reshape(np.sum(np.reshape(np.array(A_hat_conj)* np.array(e_hat_conj),(m,n_new)),axis = 1),(-1,1)) + b_hat_conj
            lower = (rho * np.array(corr_A)) + 1.0
            init_psi_hat_conj = upper / lower
            

        diff_value = np.inf #error difference  
    
        
        return(A_hat, corr_A, b_hat_conj, init_psi_hat_conj, init_theta, init_zeta, init_gamma, 
               init_gamma_out, mask_update)
    
################################################################
        #Collects all variables into a dictionary
################################################################
    
    
    def fill_feed_dict(self,values):
        feed_dict = { }
        for i in range(self._gpus):
            feed_dict[self.psi_hat_conj[i]] =  values["psi_hat_conj"]
            feed_dict[self.theta[i]] =  values["theta"][i]
            feed_dict[self.zeta[i]] =  values["zeta"][i]
            feed_dict[self.gamma[i]] =  values["gamma"][i]
            feed_dict[self.rho[i]] =  values["rho"][0]
            feed_dict[self.gamma_val[i]] =  values["gamma_val"][0]

        return feed_dict
    
################################################################
        #Break bulky dictionary to pieces
################################################################    
    def break_A_pieces(self, A,num_gpus):
        return np.split(A,num_gpus,1)

################################################################
        #Computes objectove value
################################################################    
    
    def compute_cost(self, A,b,c,lamb):
        m,n = A.shape
        c = np.reshape(np.array(c),(m*n,1))
        c=np.real(c[::m]/m)
        return 0.5*(np.sum((np.dot(A,c) - b)**2)) + lamb*np.sum(np.abs(c))
    
################################################################
        #Bulding the graph of 1 iteration
################################################################   
    
    def _graph(self, init_A, init_b, init_A_hat, init_corr_aa, init_b_hat_conj, init_theta, init_zeta, 
               init_gamma, init_gamma_out, init_lamb, init_qfac, init_rho, init_gamma_val, init_mask_update):

        self.psi_hat_conj = [None]*self._gpus
        self.theta = [None]*self._gpus
        self.zeta = [None]*self._gpus
        self.gamma = [None]*self._gpus
        self.gamma_out = [None]*self._gpus
        self.rho = [None]*self._gpus
        self.gamma_val = [None]*self._gpus

        self.theta_o = [None]*self._gpus
        self.zeta_o = [None]*self._gpus
        self.gamma_o = [None]*self._gpus
        self.gamma_out_o = [None]*self._gpus
        self.rho_o = [None]*self._gpus
        self.gamma_val_o = [None]*self._gpus
        self.upper_o = [None]*self._gpus

        n_new = int(init_A.shape[1]/self._gpus)
        m_new = int(init_A.shape[0])
        
        graph = tf.Graph()
        with graph.as_default():      
            for i in range(self._gpus):
                print("################################################################")
                with tf.device('/gpu:' + str(i)):           
                    with tf.variable_scope('constants_'+str(i)):
                        print("Transfering Constant Data to GPU" + str(i), end="")
                        A_hat = tf.constant(init_A_hat[i],tf.complex64, name='A_hat')
                        mask_update = tf.constant(init_mask_update[i],tf.float32, name = 'mask')
                        lamb = tf.constant(init_lamb,tf.float32, name = 'lambda')
                        qfac = tf.constant(init_qfac, name = 'Nesterovs_qfac')
                        corr_A = tf.constant(init_corr_aa, tf.float32, name='corr_aa')
                        b = tf.constant(init_b,tf.float32, name = 'b')
                        b_hat_conj = tf.constant(init_b_hat_conj, tf.complex64, name='b_hat_conj')
                        print(": Done.")
                        
                    with tf.variable_scope('variables_inputs_pl_'+str(i)):
                        print("Initlization Place Holders on GPU"  + str(i), end="")
                        self.psi_hat_conj[i] = tf.placeholder(tf.complex64, name = 'pl_psi_hat_conj')
                        self.theta[i] = tf.placeholder(tf.float32, name = 'pl_theta')
                        self.zeta[i] = tf.placeholder(tf.float32,  name = 'pl_zeta')
                        self.gamma[i] = tf.placeholder(tf.float32, name = 'pl_gamma')
                        self.rho[i] = tf.placeholder(tf.float32, name = 'pl_rho')            
                        self.gamma_val[i] = tf.placeholder(tf.float32, name = 'pl_gamma_val')
                        print(": Done.")
                        
                    with tf.variable_scope('variables_outputs_pl_'+str(i)):  
                        self.theta_o[i] = self.theta[i] 
                        self.zeta_o[i] = self.zeta[i] 
                        self.gamma_o[i] = self.gamma[i] 
                        self.rho_o[i] = self.rho[i] 
                        self.gamma_val_o[i] = self.gamma_val[i]    
                        
                print("Generating Compute Graph on GPU"  + str(i), end="")
                #theta_o update
                with tf.name_scope('Theta_Update'):
                    temp = tf.real(tf.reshape(tf.fft(tf.transpose(tf.multiply(A_hat, self.psi_hat_conj[i]))),(m_new*n_new,1)))
                    self.theta_o[i] = self.zeta_o[i]-self.gamma_o[i]/self.rho_o[i]- temp
                    self.theta_o[i] = self.theta_o[i] * mask_update

                #zeta_o update
                with tf.name_scope('Zeta_Update'):
                    self.zeta_o[i] = temp+(self.theta_o[i])+(self.gamma_o[i]/self.rho_o[i])
                    self.zeta_o[i] = tf.minimum(lamb,tf.abs(self.zeta_o[i])) * tf.sign(self.zeta_o[i])
    
    
                #gamma_o update
                with tf.name_scope('Gamma_Update'):
                    gamma0_o=self.gamma_o[i];
                    self.gamma_o[i] = (self.gamma_o[i] + (self.gamma_val_o[i]*self.rho_o[i])*( temp + (self.theta_o[i])-self.zeta_o[i]))
                    self.gamma_o[i]=(1+qfac)*self.gamma_o[i]-qfac*gamma0_o; 
                    tf.summary.histogram('Gamm_Output', self.gamma_o[i])

                #Psi update
                with tf.name_scope('Upper_update_for_CPUPsi'):
                    e = tf.cast(self.rho_o[i]*self.zeta_o[i]-self.rho_o[i]*self.theta_o[i]-self.gamma_o[i],tf.complex64)
                    e_hat_conj  = tf.transpose(tf.ifft(tf.reshape(e,(n_new,m_new))))  
#                     self.upper_o[i] = tf.reshape(tf.reduce_sum(A_hat_conj* e_hat_conj,1),[m_new,1])
                    self.upper_o[i] = tf.reshape(tf.reduce_sum(tf.conj(A_hat)/m_new* e_hat_conj,1),[m_new,1])

                print(": Done.")
                    
                    
                self.merged = tf.summary.merge_all()
                
        print("################################################################")
   
        outputs = {"theta": self.theta_o,
                    "zeta": self.zeta_o,
                    "gamma": self.gamma_o,
                    "rho": self.rho_o,
                    "gamma_val": self.gamma_val_o,
                    "upper": self.upper_o,
                  }

        return graph,outputs    
    
################################################################
                #Solving the Lasso
################################################################         

    def solve(self, A, b, reg_param, max_iters, threshold, 
                                  qfac, rho, learning_rate, 
                                  gamma_val, gamma_factor,inc_iter, upper_limit_rho):
        
        
        (A_hat, corr_A, b_hat_conj, init_psi_hat_conj, theta, zeta, gamma, gamma_out, 
         mask_update) = self._initialize(A, b, rho)        
        #Done initlization

        


        print('Graph Generation ...')    
        graph, outputs_nodes = self._graph(A, b, A_hat, corr_A, b_hat_conj, 
                                           theta, zeta, gamma, gamma_out, reg_param, 
                                           qfac, rho, gamma_val, mask_update)
        
        
        init_values = {"psi_hat_conj": init_psi_hat_conj,
                        "theta": theta,
                        "zeta": zeta,
                        "gamma": gamma,
                        "rho": [rho],
                        "gamma_val": [gamma_val],
                        }
        
        with tf.Session(graph=graph) as sess:
            sess.run(tf.global_variables_initializer())
            iteration_writer = tf.summary.FileWriter('./logs/1', sess.graph)
            
            # first iteration:
            feed_dict = self.fill_feed_dict(init_values) 
            output_results = sess.run(outputs_nodes, feed_dict=feed_dict)            
            cost = [self.compute_cost(A,b,output_results["gamma"],reg_param)]
            print("+ Iter: ", 1, 'Cost: ', cost[-1])
                
            # CPU for each upper and rho get phi_hat_conj for all gpus
            sum_upper = np.zeros_like(output_results["upper"][0]) + b_hat_conj
            for upper in output_results["upper"]:
                sum_upper += upper
            
            lower = (output_results["rho"][0]*corr_A) + 1.0
            output_results["psi_hat_conj"] = sum_upper / lower
            
################################################################
                #Main Loop
################################################################  
            start_time = time.time()
            for iteration in range(2,max_iters+1):
                feed_dict = self.fill_feed_dict(output_results)
                summary_new, output_results = sess.run([self.merged,outputs_nodes], feed_dict=feed_dict)
                iteration_writer.add_summary(summary_new,iteration)
                
                # CPU for each upper and rho get phi_hat_conj for all gpus
                sum_upper = np.zeros_like(output_results["upper"][0]) + b_hat_conj
                for upper in output_results["upper"]:
                    sum_upper += upper
                lower = (np.array(output_results["rho"][0])*corr_A)+ 1.0
                output_results["psi_hat_conj"] = sum_upper / lower

                if iteration % 1 == 0:
                    cost.append(self.compute_cost(A,b,output_results["gamma"],reg_param))  
                    print("+ Iter: ", iteration, 'Cost: ', cost[-1])
                    
                    if(np.std(np.array(cost[-5:])) < threshold):
                        break
                        
                    if  iteration % inc_iter == 0:
                        if(output_results["rho"][0] <= upper_limit_rho):
                            output_results["rho"] = [output_results["rho"][0]*learning_rate] * self._gpus
  
                    
        elapsed_time = time.time() - start_time
        print('Total time in seconds: ', elapsed_time)
        return output_results

## Main Code

- Specify the number of GPUs as input to TFSolver(.)


In [9]:
if __name__ == '__main__':  
    #initiaze opt parameters
    (max_iters, threshold, init_qfac, init_rho, learning_rate, init_gamma_val, 
     gamma_factor, inc_iter, upper_limit_rho) = opt_parameters()

    #test your code
    solver = TFSolver(gpus = 1)
    output_results = solver.solve(A, b, reg_param, max_iters, threshold, 
                                  init_qfac, init_rho, learning_rate, 
                                  init_gamma_val, gamma_factor,inc_iter, upper_limit_rho)

Graph Generation ...
################################################################
Transfering Constant Data to GPU0: Done.
Initlization Place Holders on GPU0: Done.
Generating Compute Graph on GPU0: Done.
################################################################
+ Iter:  1 Cost:  2.5
+ Iter:  2 Cost:  0.466960894456
+ Iter:  3 Cost:  0.111377564381
+ Iter:  4 Cost:  0.00388653625561
+ Iter:  5 Cost:  0.00557255851492
+ Iter:  6 Cost:  0.00732937397269
+ Iter:  7 Cost:  0.00247674659784
+ Iter:  8 Cost:  0.00233350916786
+ Iter:  9 Cost:  0.00166167073158
+ Iter:  10 Cost:  0.00143436536159
+ Iter:  11 Cost:  0.00182500788552
+ Iter:  12 Cost:  0.00125435554097
+ Iter:  13 Cost:  0.00128535423835
+ Iter:  14 Cost:  0.00132887283472
+ Iter:  15 Cost:  0.00152416719889
+ Iter:  16 Cost:  0.000995299385975
+ Iter:  17 Cost:  0.000942941050181
+ Iter:  18 Cost:  0.000932455477586
+ Iter:  19 Cost:  0.0012649946578
+ Iter:  20 Cost:  0.00125366462538
+ Iter:  21 Cost:  0.001212820

+ Iter:  221 Cost:  0.000500751740858
+ Iter:  222 Cost:  0.000501902012054
+ Iter:  223 Cost:  0.00050284563158
+ Iter:  224 Cost:  0.000503129113147
+ Iter:  225 Cost:  0.000502662638507
+ Iter:  226 Cost:  0.000501713304664
+ Iter:  227 Cost:  0.000500726619406
+ Iter:  228 Cost:  0.000500098999933
+ Iter:  229 Cost:  0.000499999585132
+ Iter:  230 Cost:  0.00050032952198
+ Iter:  231 Cost:  0.000500810879572
+ Iter:  232 Cost:  0.000501145091366
+ Iter:  233 Cost:  0.000501159324862
+ Iter:  234 Cost:  0.000500867904203
+ Iter:  235 Cost:  0.000500438968872
+ Iter:  236 Cost:  0.000500089820528
+ Iter:  237 Cost:  0.00049997612442
+ Iter:  238 Cost:  0.000500122861892
+ Iter:  239 Cost:  0.00050042863656
+ Iter:  240 Cost:  0.000500728732756
+ Iter:  241 Cost:  0.000500879105431
+ Iter:  242 Cost:  0.000500820373628
+ Iter:  243 Cost:  0.00050059323734
+ Iter:  244 Cost:  0.000500306810503
+ Iter:  245 Cost:  0.000500077947074
+ Iter:  246 Cost:  0.000499978180986
+ Iter:  247 Cost

In [6]:
final_result = np.reshape(np.array(output_results['gamma'])/m,(m*n,1))
final_result = np.real(final_result[::m])
ind_low = final_result < 1e-8
final_result[ind_low] = 0
print(final_result) #sparse codes

[[ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.04999308]]
